In [9]:
import os
# os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
from dotenv import load_dotenv

In [10]:
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
load_dotenv()
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

In [31]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI

In [12]:
doc_reader = PyPDFLoader("The_3_Alarms.pdf")
# doc_reader = PyPDFLoader("doc.pdf")
doc_reader

In [13]:
pages = doc_reader.load_and_split()
pages[0]

Document(page_content='THE 3 ALARMSA Simple System to Transform Your Health, Wealth, and Relationships ForeverERIC PARTAKER', metadata={'source': 'The_3_Alarms.pdf', 'page': 1})

In [14]:
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1500,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

In [15]:
texts = text_splitter.split_documents(pages)
print(texts[0])

page_content='THE 3 ALARMSA Simple System to Transform Your Health, Wealth, and Relationships ForeverERIC PARTAKER' metadata={'source': 'The_3_Alarms.pdf', 'page': 1}


In [20]:
persist_directory = 'db'

In [19]:
embeddings = OpenAIEmbeddings()

In [25]:
vectorDb = Chroma.from_documents( texts,embeddings)

C:\Users\rajan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\onnxruntime\capi\onnxruntime_validation.py:26: UserWarning: Unsupported Windows version (11). ONNX Runtime supports Windows 10 and above, only.
  warnings.warn(


In [26]:
vectorDb.persist()
vectorDb = None

In [27]:
vectorDb = Chroma(persist_directory=persist_directory,embedding_function=embeddings)

In [34]:
retriver = vectorDb.as_retriever()
llm = OpenAI(temperature=0.3)

In [35]:

retrievalQA = RetrievalQA.from_llm(llm, retriever=retriver)

In [36]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever

In [38]:
metadata_field_info = [
    AttributeInfo(
        name="book",
        description="The description of book",
        type="string",
    ),
]
document_content_description = "Talk about the book: The 3 Alarms in brief"

In [40]:
retriever = SelfQueryRetriever.from_llm(
    llm, vectorDb, document_content_description, metadata_field_info, verbose=True
)

In [42]:
retriever.get_relevant_documents("Tell me about the book")

[]